In [56]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import HTML, Javascript, display, clear_output

#vocational independent variables
pay_terms = 0
economy = 0
v_terms = 0
isa_pay = []
loan_pay = []

In [57]:

# info tab
info = widgets.HTML(value=
                     '''
                     ******************************<b> ISA Vocational Terms </b>******************************
                     <br/>
                     <br/>
                     <b>How it works</b><br/> 
                     This terms generator lets you view a sample ISA term for a trade or vocational program.
                     Pulling data from 
                     <br/>
                     <br/>
                     <b>How are payments decided?</b>
                     <br/>
                     Students can occupy 7 different statuses
                     <br/>
                     0. Not Graduated<br/>
                     1. Grace Period<br/>
                     2. In Repayment<br/>
                     3. In Deferment<br/>
                     4. In Delinquency<br/>
                     5. End (Term conditions reached)<br/>
                     6. End (Dropped out or Defaulted)<br/>
                     <br/>
                     Using external data (mainly from the census bureau) to initialize income distributions
                     and rates of delinquency, this simulation will transition students
                     between statuses every month according to their current income,
                     income share as well as their specific ISA term conditions. 
                     <br/>
                     <br/>
                     <b>How are ISA terms structured?</b>
                     <br/>
                     From the target ROI and length of pay term, the model calculates an income share that
                     would payout the return on investment over the course of the pay term. It does not
                     account for defaults, delinquencies or dropouts so the actual returns are always 
                     smaller than the target ROI.
                     <br/>
                     <br/>
                     <b>What assumptions are baked in?</b>
                     <br/>
                     i). Monthly likelihood of delinquency starts at a baseline level of 5% and increases by a 
                     burden factor that takes into account income share (higher income shares = higher burden), 
                     proximity to reaching any of the end conditions (closer = less burden) as well
                     as job market (recession = higher burden).
                     <br/>
                     <br/>
                     ii). Income growth is simulated as discrete occurences rather than a continuous rise.
                     Every year, the model will randomly increase the income of X% of students by the 
                     income growth rate set by the user. The percentage of students whose income has 
                     increased is determined by the job market variable as follows:<br/>
                         Recession = 70%
                         <br/>
                         Stable = 85%
                         <br/>
                         Expansion = 95%
                         <br/>
                         <br/>
                     i). Starting income and unemployment rate is set according to a distribution 
                     that approximates data from the Census Bureau.
                     
                     ''', layout=widgets.Layout(width='600px'))

In [58]:
# widget layout and reset button
layout = widgets.Layout(width='400px', height='40px')

reset_widget = widgets.Button(description='Reset', disabled=False,
                               button_style='Danger', icon='circle-o-notch')

# vocational school widget list
output_init_v = widgets.Output()
output_runsim_v = widgets.Output()

vocschool_drop = widgets.Dropdown(options=[('Certificate', 0),('Associates Degree', 1)],
                                  description='Level:', disabled=False, layout=layout)
vocfield_drop = widgets.Dropdown(options=[('Construction Management', 0),('Nursing', 1),('Electrician', 2),
                                          ('Plumbing', 3),('Maintenance Technician', 4)],
                                 description='Field:', disabled=False, layout=layout)
econ_widget_voc = widgets.Dropdown(options=[('Recession', 0), ('Stable', 1), ('Expansion', 2)],
                                   description='Job Market:', value=1, disabled=False, layout=layout)
payterms_widget_voc = widgets.IntSlider(min=2, max=12, step=1, description='TermsYrs:', value=5, layout=layout)
paycap_widget_voc = widgets.FloatSlider(min=1.0, max=2.5, step=0.25, description='PayCap: ', value=2.0,
                                    readout_format='.0%', layout=layout)
expm_widget_voc = widgets.FloatSlider(min=0.0, max=1.5, step=0.1, description='TargetROI: ', value=0.5,
                                  readout_format='.0%', layout=layout)

# loan widgets
loantype_drop = widgets.Dropdown(options=[('Fed. Undergrad', 0.0453),('Fed. Grad', 0.0608),
                                          ('Fed. PLUS', 0.0708), ('Private w/ Cosigner', 0.0785),
                                          ('Private w/o Cosigner', 0.1205)],
                                 description='Loan Type:', disabled=False, layout=layout)

loanterm_widget = widgets.IntSlider(min=5, max=15, step=1, description='LoanTerm:', value=10, layout=layout)

# buttons and boxes
init_widget_voc = widgets.Button(description='Generate Terms', disabled=False,
                               button_style='success', icon='cog')
loan_widget = widgets.Button(description='Compare with a Loan', disabled=False,
                             button_style='success', icon='play',
                             layout=widgets.Layout(width='200px'))

voc_list_widget = [vocschool_drop, vocfield_drop, econ_widget_voc, payterms_widget_voc, 
                   paycap_widget_voc, expm_widget_voc, widgets.HBox([init_widget_voc, reset_widget])]

loan_box = widgets.VBox([loantype_drop, loanterm_widget, widgets.HBox([loan_widget, reset_widget])])

vocation_box = widgets.VBox(voc_list_widget)

In [59]:
sim_tab = widgets.Tab(children=[info, vocation_box])
sim_tab.set_title(0, 'Description')
sim_tab.set_title(1, 'Vocational Terms')

In [60]:
# vocational term functions

def voc_init(v_level, v_field):
    if v_level == 0: # certificate
        if v_field == 0: # construction management
            return (10000 , 57000, 76500, 86200)
        elif v_field == 1: # nursing
            return (5000 , 23400, 25700, 27200)
        elif v_field == 2: # electrician
            return (3500 , 27500, 40200, 54500)
        elif v_field == 3: # plumbing
            return (6000 , 32800, 59800, 68200)
        elif v_field == 4: # maintenance technician
            return (3500 , 38400, 46250, 48800)
    else: # associates degree
        if v_field == 0: # construction management
            return (3500 ,70000, 99999, 99999)
        elif v_field == 1: # nursing
            return (3500 , 56000, 67000, 70600)
        elif v_field == 2: # electrician
            return (3500 ,70000, 99999, 99999)
        elif v_field == 3: # plumbing
            return (3500 , 35000, 60100, 69200)
        elif v_field == 4: # maintenance technician
            return (3500, 70000, 99999, 99999)

def igr_calc(start_inc, mid_inc, end_inc):
    igr_early = ((mid_inc/start_inc)**(1/5))-1
    igr_late =((end_inc/mid_inc)**(1/5))-1
    igr_ave = (igr_early+igr_late)/2
    return (igr_early, igr_ave)
    
def inc_threshold(inc_share):
    mincome = 30000/(1-inc_share)
    return mincome

def field_str(v_field):
    if v_field == 0: # construction management
        return 'Construction Mgt'
    elif v_field == 1: # nursing
        return 'Nursing'
    elif v_field == 2: # electrician
        return 'Electrician'
    elif v_field == 3: # plumbing
        return 'Plumbing'
    elif v_field == 4: # maintenance technician
        return 'Maintenance Technician'

def level_str(level):
    if level == 0:
        return 'Certificate'
    else:
        return 'Associate Deg.'
    
#def convert_total():
    
# vocational plot functions
def repay_schedule(start_inc, inc_share, igr, incthresh, maxowed):
    inc_arr = []
    inc = start_inc
    for i in range(int(v_terms*2)):
        inc_arr.append(inc)
        if (i%12) == 0:
            inc = income_growth(inc, igr)
            
    pay_array = [(x*inc_share)/12 for x in inc_arr]
    monthly_pay_array = [0, 0, 0, 0, 0, 0]
    counter = 0
    total_paid = 0
    for i in range(len(inc_arr)):
        if counter >= v_terms:
            return monthly_pay_array
        elif inc_arr[i] < incthresh:
            monthly_pay_array.append(0)
        else: 
            ahead_pay = total_paid + pay_array[i]
            if total_paid >= maxowed:
                return monthly_pay_array
            elif ahead_pay >= maxowed:
                monthly_pay_array.append((maxowed-total_paid))
                return monthly_pay_array
            else:
                monthly_pay_array.append(pay_array[i])
                total_paid += pay_array[i]
                counter += 1
                
    return monthly_pay_array

def loan_repay_schedule(princ, intrate, loanterm):
    ir = intrate/12
    mp = abs(npf.pmt(ir, loanterm, princ))
    loan_pay_array = [mp for i in range(loanterm)]
    return loan_pay_array

def plot_repay_schedule(mpa, lpa):
    print('\n\n\nMonthly Payments Graph')
    while len(mpa) > len(lpa):
        lpa.append(0)
    while len(mpa) < len(lpa):
        mpa.append(0)
    
    fig = plt.figure(num=None, figsize=(8,6), dpi=80)
    ax = fig.add_subplot(111)
    ax.plot(np.arange(len(mpa)), mpa, label='ISA')
    ax.plot(np.arange(len(mpa)), lpa, label='Student Loan')
    ax.yaxis.set_major_formatter('${x:1.2f}')
    ax.set_xlabel('Months')
    ax.set_ylabel('Repayment')
    plt.legend(loc=2)
    print('\n\n ISA Total Pay: ','${:,.0f}'.format(sum(mpa)))
    print('\n\n Loan Total Pay: ','${:,.0f}'.format(sum(lpa)))
    plt.show()

In [61]:
def income_growth(curr_inc, growth_rate):
    nogrowth = 0
    if economy == 0:
        nogrowth = 0.30
    elif economy == 1:
        nogrowth = 0.15
    elif economy == 2:
        nogrowth = 0.05
    payraise = np.random.choice([0,1], p=[nogrowth, 1-nogrowth])
    if payraise == 0:
        return curr_inc
    if payraise == 1:
        return curr_inc*(1+growth_rate)
    
def income_share(dis_amt, start_inc, pay_terms, exp_payout_multiple, igr):
    inc_arr = []
    years = int(pay_terms/12)
    inc = start_inc
    for i in range(years):
        inc_arr.append(inc)
        inc = income_growth(inc, igr)
        
    total_inc = np.sum(inc_arr)    
    mean_yr_inc = total_inc/years
    exp_payout = dis_amt*exp_payout_multiple
    
    yearly_pay = exp_payout/years
    income_sh = yearly_pay/mean_yr_inc
    return income_sh

In [62]:
def reset_run_all(ev):
    output_init_v.clear_output(wait=False)
    output_runsim_v.clear_output(wait=False)
    display(Javascript('IPython.notebook.execute_all_cells()'))

def voc_gen(b=None):
    global v_terms, loan_pay, isa_pay, economy
    
    v_level = vocschool_drop.value
    v_field = vocfield_drop.value
    v_terms = payterms_widget_voc.value*12
    cap = paycap_widget_voc.value
    target_roi = expm_widget_voc.value+1
    economy = econ_widget_voc.value
    loan_ir = loantype_drop.value
    loan_term = loanterm_widget.value*12
    
    disb, st_inc, mid_inc, end_inc = voc_init(v_level, v_field)
    igr_early, igr_av = igr_calc(st_inc, mid_inc , end_inc)
    igr = igr_early
    if v_terms >= 50:
        igr = igr_av
        
    incshare = income_share(disb, st_inc, v_terms, target_roi, igr)
    incthresh = inc_threshold(incshare)
    isa_pay = repay_schedule(st_inc, incshare, igr, incthresh, cap*disb)
    loan_pay = loan_repay_schedule(disb, loan_ir, loan_term)
    student_df = pd.DataFrame(columns=['Program Type', 'Field of Study', 'Exp. 1Yr Income',
                                        'Exp. 5Yr Income', 'Exp. 10Yr Income', 
                                        'Annualized Inc. Growth'])
    student_df.loc[0] = [level_str(v_level), field_str(v_field), '${:,.0f}'.format(st_inc),
                         '${:,.0f}'.format(mid_inc), '${:,.0f}'.format(end_inc), '{:.1%}'.format(igr_av)]
    student_df = student_df.rename(index={0:''})
    vocterms_df = pd.DataFrame(columns=['Disbursement Amount', 'Income Share', 'Minimum Income',
                                            'Pay Terms', 'Exp. Terms', 'Max Owed', 'Expected Total Payment'])
    vocterms_df.loc[0] = ['${:,.0f}'.format(disb), '{:.2%}'.format(incshare), 
                          '${:,.0f}'.format(incthresh), int(v_terms), int(v_terms*2),
                          '${:,.0f}'.format(cap*disb), '${:,.0f}'.format(sum(isa_pay))]
    vocterms_df = vocterms_df.rename(index={0:''})
    
    with output_init_v:
        print('\n\n\nStudent Profile')
        display(student_df)
        print('\n\n\nISA Terms')
        display(vocterms_df)
        display(loan_box)
def show_compare(b=None):
    with output_runsim_v:
        plot_repay_schedule(isa_pay, loan_pay)
        display(reset_widget)

In [63]:
# button function calls
init_widget_voc.on_click(voc_gen)
loan_widget.on_click(show_compare)


# display widgets
display(sim_tab)
display(output_init_v)
display(output_runsim_v)

Output()

Output()